In [2]:
import pandas as pd
df = pd.read_csv('../data/example.csv',sep='\t')
documents = list(df['Q'])
ids = list(df['id'].astype(str))

In [5]:
import pypinyin

def chn_to_pinyin(chn):
    return pypinyin.slug(chn[6], separator='')

df['Day_Stem'] = df['Time'].apply(chn_to_pinyin)
df.to_csv('../data/example.csv',sep='\t',index=False)

In [15]:
def extract_gua_id(raw_text):
    gua_id = raw_text.split(":")[0]
    return gua_id

df['Primary_id'] = df['Primary'].apply(extract_gua_id)
df['Alter_id'] = df['Alter'].apply(extract_gua_id)

In [18]:
df.drop(columns=['Primary', 'Alter']).to_csv('../data/example_translated.csv', sep='\t', index=False)

In [2]:
print(type(ids))

<class 'list'>


In [3]:
import chromadb
client = chromadb.Client()
collection = client.get_or_create_collection(name="iching")
# switch `add` to `upsert` to avoid adding the same documents every time
collection.add(ids=ids, documents=documents)

In [8]:
results = collection.query(
    query_texts=["Will I get a promotion in the next year?"], # Chroma will embed this for you
    n_results=2 # how many results to return
)
print(results['documents'])

[['Will my salary increase after starting the new job?', 'When will I gain financial benefits?']]


In [ ]:
collection.sea